In [89]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from scrapp_fonction import *
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from dict import months

In [90]:
URL = 'https://www.lequipe.fr/Football/ligue-1/saison-2020-2021/page-calendrier-resultats'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
results = soup.find_all('select', class_='SelectNav__select')

In [91]:
list_url_journey = []
for result in results:
    liens_journey = result.find_all('option')
    for liens in liens_journey:
        list_url_journey.append(liens.get('value'))

In [92]:
row_list = []
for url_journey in list_url_journey:
    journey = pd.read_json(url_journey, lines=True)
    for match in journey['items'][0]:
        try:
            if len(match['items']) == 1:
                match_dict={}
                match_dict['place'] = match['items'][0]['event']['lieu']['ville']
                match_dict['date'] = match['items'][0]['event']['date']
                match_dict['dom_name'] = match['items'][0]['event']['specifics']['domicile']['equipe']['nom']
                match_dict['ext_name'] = match['items'][0]['event']['specifics']['exterieur']['equipe']['nom']
                match_dict['but_dom'] = match['items'][0]['event']['specifics']['score']['domicile']
                match_dict['but_ext'] = match['items'][0]['event']['specifics']['score']['exterieur']
                match_dict['winner'] = match['items'][0]['event']['specifics']['vainqueur']
                match_dict['lien'] = match['items'][0]['event']['lien_web']
                row_list.append(match_dict)
            elif len(match['items']) > 1:
                for m in range(len(match['items'])):
                    match_dict={}
                    match_dict['place'] = match['items'][m]['event']['lieu']['ville']
                    match_dict['date'] = match['items'][m]['event']['date']
                    match_dict['dom_name'] = match['items'][m]['event']['specifics']['domicile']['equipe']['nom']
                    match_dict['ext_name'] = match['items'][m]['event']['specifics']['exterieur']['equipe']['nom']
                    match_dict['but_dom'] = match['items'][m]['event']['specifics']['score']['domicile']
                    match_dict['but_ext'] = match['items'][m]['event']['specifics']['score']['exterieur']
                    match_dict['winner'] = match['items'][m]['event']['specifics']['vainqueur']
                    match_dict['lien'] = match['items'][m]['event']['lien_web']
                    row_list.append(match_dict)
        except:
            pass

data = pd.DataFrame(row_list)


## dataframe Match

### Partie : date

In [93]:
pd.set_option('display.max_rows', None)

In [94]:
df_match = data[['date', 'place']]

In [95]:
dates = []
month = []
day = []
year = []

for e in df_match.date:
    date_str = e[0:10]
    date_str = date_str.replace("-", "/")
    year_str = e[0:4]
    month_str = e[5:7]
    day_str = e[8:10]
    year.append(year_str)
    month.append(month_str)
    day.append(day_str)

In [96]:
df_days = pd.DataFrame(day)
df_days['day'] = df_days
dfdays = df_days['day']
df_match[['day']] = pd.Series([dfdays])

df_months = pd.DataFrame(month)
df_months['month'] = df_months
dfmonths = df_months['month']
df_match[['month']] = pd.Series([dfmonths])

df_years = pd.DataFrame(year)
df_years['year'] = df_years
dfyears = df_years['year']
df_match[['year']] = pd.Series([dfyears])

df_match.month = df_match.month.astype(str)

In [98]:
thisdict = {
  "Lille": "nord-pas-de-calais",
  "Paris": "ile-de-france",
  "Monaco": 'provence-alpes-c-te-d-azur',
  "Lyon": "rh-ne-alpes",
  "Marseille": "provence-alpes-c-te-d-azur",
  "Rennes": 'bretagne',
  "Lens": "nord-pas-de-calais",
  "Montpellier": "languedoc-roussillon",
  "Nice": 'provence-alpes-c-te-d-azur',
  "Metz": "lorraine",
  "Saint-Étienne": "rh-ne-alpes",
  "Bordeaux": 'aquitaine',
  "Angers": "pays-de-la-loire",
  "Reims": "champagne-ardenne",
  "Strasbourg": 'alsace',
  "Lorient": "bretagne",
  "Brest": "bretagne",
  "Nantes": "pays-de-la-loire",
  "Nimes": 'languedoc-roussillon',
  "Dijon": 'bourgogne',
}

In [12]:
city = df_match.place
city = city.str.replace('î','i')
mois_chiffre = df_match.month
year = df_match.year
day = df_match.day

### Partie : région

In [99]:
regions = []
list_urls = []

for e in city:
    region = thisdict[e]
    regions.append(region)
    
df_regions = pd.DataFrame(regions)
df_regions['region'] = df_regions
dfregions = df_regions['region']
df_match[['region']] = pd.Series([dfregions])
region = df_match.region 
city = city.str.lower()

In [122]:
### Partie : météo - température et pluie

In [ ]:
for region, city, year, mois_chiffre, day in zip(df_match['region'], city, year,mois_chiffre, day):
    url = "https://www.historique-meteo.net/france/"+ region + '/' + city + '//' + year + '/' + mois_chiffre + '/' + day + '//'
    list_urls.append(url)

In [103]:
temperature = []
rainfall = []

In [104]:
for url in list_urls:
    try:
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        table = soup.find_all('table')
        df = pd.read_html(str(table))
        df = df[0]
        df = df.rename(columns={df.columns[3]: 'Date_day'})
        Précipitations = df[df['Unnamed: 0'] == "Précipitations"]
        Précipitation = Précipitations.Date_day
        Précipitation = str(Précipitation)
        Précipitation = Précipitation[2:8]
        Précipitation = Précipitation.strip()
        if Précipitation == 'ries([':
            Précipitation = None
        Précipitation = int(''.join(i for i in Précipitation if i.isdigit()))
        rainfall.append(Précipitation)
    except Exception:
        rainfall.append(None)
    try:
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        table = soup.find_all('table')
        df = pd.read_html(str(table))
        df = df[0]
        df = df.rename(columns={df.columns[3]: 'Date_day'})
        Température = int(df['Date_day'].head(1)[0].split('°')[0])
        temperature.append(Température)
    except Exception:
        temperature.append(None)

In [105]:
# création de la colonne rainfall
df_rainfall = pd.DataFrame(rainfall)
df_rainfall['rainfall'] = df_rainfall
dfrainfall = df_rainfall['rainfall']
df_match[['rainfall']] = pd.Series([dfrainfall])

# création de la colonne temperature
df_temperature = pd.DataFrame(temperature)
df_temperature['temperature'] = df_temperature
dftemperature = df_temperature['temperature']
df_match[['temperature']] = pd.Series([dftemperature])

In [107]:
df_match.rainfall.fillna(df_match.rainfall.mean(), inplace=True)
df_match.temperature.fillna(df_match.temperature.mean(), inplace=True)

In [503]:
df_match

,date,place,day,month,year,region,rainfall,temperature
0,2020-08-21T19:00:00+0200,Bordeaux,21,08,2020,aquitaine,5.000000,24.000000
1,2020-08-22T17:00:00+0200,Dijon,22,08,2020,bourgogne,19.000000,26.000000
2,2020-08-22T21:00:00+0200,Lille,22,08,2020,nord-pas-de-calais,1.000000,21.000000
3,2020-08-23T13:00:00+0200,Monaco,23,08,2020,provence-alpes-c-te-d-azur,1.000000,27.000000
4,2020-08-23T15:00:00+0200,Lorient,23,08,2020,bretagne,1.000000,19.000000
5,2020-08-23T15:00:00+0200,Nîmes,23,08,2020,languedoc-roussillon,4.277397,31.000000
6,2020-08-23T17:00:00+0200,Nice,23,08,2020,provence-alpes-c-te-d-azur,1.000000,27.000000
7,2020-09-15T21:00:00+0200,Montpellier,15,09,2020,languedoc-roussillon,4.277397,26.000000
8,2020-09-16T21:00:00+0200,Paris,16,09,2020,ile-de-france,1.000000,32.000000
9,2020-09-17T21:00:00+0200,Marseille,17,09,2020,provence-alpes-c-te-d-azur,4.277397,27.000000


In [115]:
#data  = data_match.rename(columns={"TimeStamp": "created_at"})
#data.insert(0, 'updated_at', pd.to_datetime('now').replace(microsecond=0))
#match_table_df = data[['date', 'place', 'rainfall', 'temperature', 'created_at', 'updated_at']]
#match_table_df['date'] = pd.to_datetime(match_table_df['date'])
#match_table_df.to_csv('match_table_final.csv')

In [114]:
df_match['created_at'] = datetime.now()
df_match['updated_at'] = datetime.now()

## Dataframe  goals

### Partie : buteurs, date et ville

In [117]:
list_liens_matchs = data['lien'].tolist()
#list_liens_matchs

In [118]:
buteurs = []
dates = []
match_places = []

In [119]:
for match in list_liens_matchs:
    req = requests.get(match)
    soup = BeautifulSoup(req.content, 'html.parser')
    goals = soup.find_all('div', class_="Scoreboard__goals")
    
    # on récupère les buteurs des 2 équipes
    for buteur in goals:
        buteur = buteur.text.split()
        buteurs.append(buteur)
        
        # on récupère les lieus ou le match se jouent   
        match_place = soup.find('div', class_="TeamScore__identity")
        match_place = match_place.text
        match_place = match_place.strip()
        match_places.append(match_place)
            
        # on récupère la date du match
        date = soup.find('div', class_="Scoreboard__titles pageHeading")
        date = date.text.split()
        if date[-1].isdigit(): 
            date = "-".join(date[-3:])
            dates.append(date)
        else:
            date = soup.find('div', class_="pageHeading pageHeading__subtitle")
            date = date.text.split()
            date = "-".join(date[1:4])
            dates.append(date)

### Partie : nom des joueurs

In [123]:
df_buts = pd.DataFrame()
df_buts[['buteurs', 'dates', 'match_places']] = pd.Series([buteurs, dates, match_places])

In [124]:
import re
names = []
minutes = []
initial = []
carac = []
penalties = []
own_goals = []

def find_letter(text):
    txt = re.findall(r'[A-Z\]+',text)
    return " ".join(txt)

In [125]:
for minute in df_buts.buteurs:
    minute = ' '.join(minute)
    minute = int(''.join(filter(str.isdigit, minute)))
    minute = str(minute)
    minutes.append(minute)
    
df_minutes = pd.DataFrame(minutes)
df_minutes['minute'] = df_minutes
dfminute = df_minutes['minute']
df_buts[['minute']] = pd.Series([dfminute])

for e in df_buts.buteurs:
    penalty = [s for s in e if "(s.p.)" in s]
    own_goal = [s for s in e if "(csc)" in s]
    penalties.append(penalty)
    own_goals.append(own_goal)
    
df_penalties = pd.DataFrame(penalties)
df_penalties['penalty'] = df_penalties
dfpenalty = df_penalties['penalty']
df_buts[['penalty']] = pd.Series([dfpenalty])

df_own_goals = pd.DataFrame(own_goals)
df_own_goals['own_goal'] = df_own_goals
dfown_goals = df_own_goals['own_goal']
df_buts[['own_goal']] = pd.Series([dfown_goals])

In [126]:
for e in df_buts.buteurs:
    e_clean = ''.join( c for c in e if  c not in '(s.p.)' )
    e_clean = ''.join( c for c in e_clean if  c not in '(csc)' )
    str1 = ''.join(e_clean)
    res = ''.join(c for c in str1 if c.isalpha())
    name = res[0] + '. ' + res[1:]
    names.append(name)

In [130]:
df_names = pd.DataFrame(names)
df_names['full_name'] = df_names
dfname = df_names['full_name']
df_buts[['full_name']] = pd.Series([dfname])

In [133]:
#df_buts.to_csv('goals_table_clean_name.csv')

In [134]:
#df_buts